<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Consolidaci%C3%B3n_de_pedidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalaciones previas

In [154]:
#!pip install xlsxwriter
#!pip install python-calamine
#!pip install fastexcel

# Librerías necesarias

In [155]:
import polars as ps
import pandas as pd
import numpy as np

# Archivos necesarios

In [156]:
renglonaje_og = ps.read_excel("Renglonaje.xlsx", sheet_id=1)
renglonaje = renglonaje_og.select(['código', 'clave', 'Total que paga Edy a Truper contando reserva en garantía CON IVA'])
renglonaje = renglonaje.rename({'Total que paga Edy a Truper contando reserva en garantía CON IVA':'Costo', 'código':'clave', 'clave':'sku'})

## Relación Asín-SKU

In [157]:
asin = ps.read_excel('Asin.xlsx', sheet_id=2)
asin = asin.rename({'sku':'alfa', 'Sku Truper':'sku'})
asin = asin.select(['sku', 'alfa', 'asin'])
# Conditionally update the 'alfa' column:
# - Use with_columns to modify existing columns.
# - ps.when checks if the value in 'alfa' is equal to the value in 'sku'.
# - .then(ps.lit('')) replaces the value with an empty string if the condition is true.
# - .otherwise(ps.col('alfa')) keeps the original 'alfa' value if the condition is false.
# - .alias('alfa') renames the resulting column back to 'alfa'.
asin = asin.with_columns(
    ps.when(ps.col('alfa') == ps.col('sku'))
    .then(ps.lit(''))
    .otherwise(ps.col('alfa'))
    .alias('alfa')
)

In [158]:
renglonaje = renglonaje.join(asin, on='sku', how='left')

In [159]:
renglonaje = renglonaje.filter(ps.col('Costo').is_not_null())
renglonaje = renglonaje.fill_null('')

In [160]:
c_sku = renglonaje.select(['sku', 'Costo'])
c_sku = c_sku.rename({'Costo':'Costo_sku'})
c_alfa = renglonaje.select(['alfa', 'Costo'])
c_alfa = c_alfa.rename({'Costo':'Costo_alfa', 'alfa':'sku'})
c_asin = renglonaje.select(['asin', 'Costo'])
c_asin = c_asin.rename({'Costo':'Costo_asin', 'asin':'sku'})
c_clave = renglonaje.select(['clave', 'Costo'])
c_clave = c_clave.rename({'Costo':'Costo_clave', 'clave':'sku'})
c_clave = c_clave.with_columns(ps.col('sku').cast(ps.Utf8))

# Reportes de Amazon
Aquí se definen cuantos reportes y transacciones se van a procesar

In [161]:
loop = True
while loop:
  try:
    reportes_cantidad = input("Cantidad de reportes: ")
    reportes_cantidad = int(reportes_cantidad)
    loop = False
  except:
    print("\nIngresa un número válido\n")

Cantidad de reportes: 3


In [162]:
loop = True
while loop:
  try:
    transacciones_cantidad = input("Cantidad de transacciones: ")
    transacciones_cantidad = int(transacciones_cantidad)
    loop = False
  except:
    print("\nIngresa un número válido\n")

Cantidad de transacciones: 57


# Leyendo Reportes

Si solo se va a leer un reporte, este deberá ser guardado como "Reporte". Si no, los reportes deberán ser guardados de forma seriada, es decir, "Reporte1", "Reporte2", "Reporte3", etc.

El orden no debería importar

In [163]:
archivo = ''
archivos = []
if reportes_cantidad == 1:
    archivo = "Reportes/Reporte.txt"
else:
  for i in range(reportes_cantidad):
      archivos.append((f"Reportes/Reporte{i+1}.txt"))

In [164]:
if archivo == '':
  reportes = ps.read_csv(archivos[0], separator="\t")
else:
  reportes = ps.read_csv(archivo, separator="\t")

In [165]:
if len(archivos) > 1:
  for i in range(1, reportes_cantidad):
      reportes = reportes.vstack(ps.read_csv(archivos[i], separator="\t", schema=reportes.schema, truncate_ragged_lines=True))
  print(f"Uniendo {reportes_cantidad} reportes")


Uniendo 3 reportes


# Leyendo Transacciones

Si solo se va a leer una transacción, esta deberá ser guardada como "Transacciones". Si no, deberán ser guardadas de forma seriada, como los reportes.

***Nota: Los reportes deberán ser pasados primero por la herramienta de limpieza de transacciones. Si no, generarán problemas***

In [166]:
archivo = ''
archivos = []
if transacciones_cantidad == 1:
    archivo = "Transacciones/Transacciones.csv"
else:
  for i in range(transacciones_cantidad):
      archivos.append((f"Transacciones/Transacciones{i+1}.csv"))

In [167]:
if archivo == '':
  transacciones = ps.read_csv(archivos[0], separator=",")
else:
  transacciones = ps.read_csv(archivo, separator=",")

In [168]:
if len(archivos) > 1:
  for i in range(1, transacciones_cantidad):
      transacciones = transacciones.vstack(ps.read_csv(archivos[i], separator = ',', schema=transacciones.schema, truncate_ragged_lines=True))

Renombramos la columna de Número de pedido a amazon-order-id, para hacer la unión luego con el reporte de pedidos

In [169]:
transacciones = transacciones.rename({'Número de pedido':'amazon-order-id'})

In [170]:
dfr = reportes.join(transacciones, on='amazon-order-id', how='left', suffix='_transacciones')

In [171]:
formato_num = {ps.Int64:"0", ps.Float64:"0.00"}

In [172]:
dft = transacciones.join(reportes, on='amazon-order-id', how='left', suffix='_reportes')

# Consolidando reporte de envíos

In [173]:
archivo_envios = 'Envios enero-julio.xlsx'
envios_1 = pd.read_excel(archivo_envios, sheet_name = 0, header=3)
envios_2 = pd.read_excel(archivo_envios, sheet_name = 1, header=3)
reporte_envios = pd.concat([envios_1, envios_2])
reporte_envios = reporte_envios.reset_index(drop=True)
reporte_envios.to_excel("Envios.xlsx", index=False)

In [174]:
envios = ps.read_excel("Envios.xlsx", sheet_id=1)

In [175]:
envios = envios.with_columns(ps.col('PROTECCION ($)').fill_null(0))
envios = envios.with_columns(ps.col('PROTECCION ($)').cast(ps.Float32))

# Juntando el reporte de envíos y las transacciones

In [176]:
envios = envios.rename({'NUMERO ORDEN':'amazon-order-id'})
envios = envios.select(['amazon-order-id', 'VALOR GUIA ($)', 'ZONA EXTENDIDA ($)', 'EXTRAGRANDE ($)', 'SOBREPESOS ($)', 'PROTECCION ($)', 'TOTAL GUIA ($)'])
dfr = dfr.join(envios, on='amazon-order-id', how='left')

In [177]:
dfr = dfr.with_columns(ps.col('VALOR GUIA ($)').fill_null(0),
                       ps.col('ZONA EXTENDIDA ($)').fill_null(0),
                       ps.col('EXTRAGRANDE ($)').fill_null(0),
                       ps.col('SOBREPESOS ($)').fill_null(0),
                       ps.col('PROTECCION ($)').fill_null(0),
                       ps.col('TOTAL GUIA ($)').fill_null(0))

In [178]:
dft = dft.join(envios, on='amazon-order-id', how='left')
dft = dft.with_columns(ps.col('VALOR GUIA ($)').fill_null(0),
                       ps.col('ZONA EXTENDIDA ($)').fill_null(0),
                       ps.col('EXTRAGRANDE ($)').fill_null(0),
                       ps.col('SOBREPESOS ($)').fill_null(0),
                       ps.col('PROTECCION ($)').fill_null(0),
                       ps.col('TOTAL GUIA ($)').fill_null(0))

In [179]:
dfr = dfr.join(c_sku, on='sku', how='left')
dfr = dfr.join(c_alfa, on='sku', how='left')
dfr = dfr.join(c_asin, on='sku', how='left')
dfr = dfr.join(c_clave, on='sku', how='left')

In [180]:
dfr = dfr.with_columns(ps.col('Costo_sku').fill_null(0),
                       ps.col('Costo_alfa').fill_null(0),
                       ps.col('Costo_asin').fill_null(0),
                       ps.col('Costo_clave').fill_null(0))

In [181]:
dfr = dfr.with_columns((ps.col('Costo_sku') + ps.col('Costo_alfa') + ps.col('Costo_asin') + ps.col('Costo_clave')).alias('Costo Truper'))

In [182]:
dfr = dfr.drop(['Costo_sku', 'Costo_alfa', 'Costo_asin', 'Costo_clave'])

In [183]:
dfr.write_excel('Reporte de envios.xlsx', worksheet="Reportes")

In [184]:
dft = dft.join(c_sku, on='sku', how='left')
dft = dft.join(c_alfa, on='sku', how='left')
dft = dft.join(c_asin, on='sku', how='left')
dft = dft.join(c_clave, on='sku', how='left')

In [185]:
dft = dft.with_columns(ps.col('Costo_sku').fill_null(0),
                       ps.col('Costo_alfa').fill_null(0),
                       ps.col('Costo_asin').fill_null(0),
                       ps.col('Costo_clave').fill_null(0))

In [186]:
dft = dft.with_columns((ps.col('Costo_sku') + ps.col('Costo_alfa') + ps.col('Costo_asin') + ps.col('Costo_clave')).alias('Costo Truper'))

In [187]:
dft = dft.drop(['Costo_sku', 'Costo_alfa', 'Costo_asin', 'Costo_clave'])

In [188]:
dft.write_excel('Reporte de transacciones.xlsx', dtype_formats={ps.Float64():'General'}, worksheet="Transacciones")